<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/main/Flairmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y torch
!pip install flair

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 34.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing 

In [72]:
import os
import random
import json
import re

file = {}
with open('annotated.json') as f:
    file = json.load(f)
    # print(file[1]['annotations'])


In [61]:
def write(filename, text):
  with open(f"{filename}.conll", "w") as file1:
    file1.write("\n\n".join(text))

In [62]:
# file_item = file[0]
def get_labels(file_item):
  rows = []
  position = 0
  next_label = 0
  # Sort the labels on start?
  labels = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'labels']
  labels.sort(key=lambda x: x['value']['start'])
  item_count = 1
  b_count = 0
  ids = {} # count -> id
  tokens = [item for item in re.split(r'(\s+)|([\(,\.\)\\;:$*&"\-–])|(\')|(\'\w+)', file_item['data']['text']) if item]
  for i, item in enumerate(tokens):
    if re.search(r'\s+', item):
      position += len(item)
      continue
    start = position
    end = position + len(item)
    row = f'{item_count}\t{item}'
    label = '\tO'

    if len(labels) > next_label:
      if start >= labels[next_label]['value']['start']:
        # if end <= labels[next_label]['value']['end']:
          # the current item has a label, and might have a relation
          id = labels[next_label]["id"]
          ids[item_count] = id
          if end > labels[next_label]['value']['end']:
            difference = end - labels[next_label]['value']['end']
            end -= difference
            tokens.insert(i+1, item[-difference:])
            item = item[:-difference]
          if start == labels[next_label]['value']['start']:
            # Current Item is start of a label
            b_count += 1
            label = '\tB-' + labels[next_label]["value"]["labels"][0]

          else:
            # Current item is inside a label
            if '\tB-' + labels[next_label]["value"]["labels"][0] in rows[-1] or '\tI-' + labels[next_label]["value"]["labels"][0] in rows[-1]:
              label = '\tI-' + labels[next_label]["value"]["labels"][0]
            else:
              b_count += 1
              label = '\tB-' + labels[next_label]["value"]["labels"][0]

      if end >= labels[next_label]['value']['end']:
        next_label += 1


    row = row + label
    position += len(item)
    item_count += 1
    rows.append(row)
  if len(labels) != b_count:
    print(f'labels: {len(labels)}, B-tags: {b_count}')
    print(labels)
    print(ids)
    print(f'missing label: {[item for item in labels if item["id"] not in ids.values()]}')
    print(rows)
  return rows, ids


In [63]:
def get_relations(file_item, ids):
  keys = ids.keys()
  relations = [item for item in file_item['annotations'][0]['result'] if item['type'] == 'relation']
  bio_relations = []
  found_relations = 0
  for relation in relations:
    from_ids = []
    to_ids = []
    rel_type = relation['labels'][0]
    for key in keys:
      if ids[key] == relation['from_id']:
        from_ids.append(key)
      if ids[key] == relation['to_id']:
        to_ids.append(key)
    from_ids = [int(i) for i in from_ids]
    from_ids.sort()
    to_ids = [int(i) for i in to_ids]
    to_ids.sort()
    if len(from_ids) > 0 and len(to_ids) > 0:
      found_relations += 1
      bio_relations.append(f'{from_ids[0]};{from_ids[-1]};{to_ids[0]};{to_ids[-1]};{rel_type}')
  if found_relations != len(relations):
    print(f'found relations: {found_relations}, actual relations: {len(relations)}')
    print(relations)
    print(bio_relations)
    print(keys)
    print(ids)
  return bio_relations

In [64]:
conllu_file = []
for item in file:
  doc = ''
  rows, ids = get_labels(item)
  relations = get_relations(item, ids)
  if relations and len(relations) > 0:
    doc += '# relations = ' + ('|'.join(relations))
  doc += '\n' + ('\n'.join(rows))
  conllu_file.append(doc)

print(len(conllu_file))
write('annotated', conllu_file)

40


In [66]:
with open('annotated.conll') as f:
  data = f.read()
  splits = data.split(2*os.linesep)
  print(len(splits))
  splits = sorted(splits, key=lambda x: random.random())

  test = splits[:20]
  dev = splits[20:]

  # write('train', train)
  write('test', test)
  write('dev', dev)

40


In [73]:
import torch

torch.cuda.is_available()

True

In [2]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {1: 'text', 2: 'ner'}
data_folder = ''

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')
print(f'Corpus size: {len(corpus.train)}')

2023-10-18 10:47:14,203 Reading data from .
2023-10-18 10:47:14,217 Train: train.conll
2023-10-18 10:47:14,218 Dev: dev.conll
2023-10-18 10:47:14,222 Test: test.conll
Corpus size: 95


In [75]:
# tag to predict
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)


2023-10-18 09:59:10,116 Computing label dictionary. Progress:


95it [00:00, 10191.29it/s]

2023-10-18 09:59:10,133 Dictionary created for label 'ner' with 6 values: Winner (seen 824 times), Date (seen 571 times), Nationality (seen 479 times), Prizetype (seen 88 times), Reason (seen 54 times)


In [76]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings, TransformerWordEmbeddings
from flair.models import SequenceTagger
from typing import List
# embedding_types : List[TokenEmbeddings] = [
#         WordEmbeddings('glove'),
        ## other embeddings
        # ]
# embeddings : StackedEmbeddings = StackedEmbeddings(
#                                 embeddings=embedding_types)

embeddings = TransformerWordEmbeddings(
    model='distilbert-base-cased',
    layers="-1",
    subtoken_pooling='first',
    fine_tune=True,
    use_context=True,
    respect_document_boundaries=False,
)

tagger: SequenceTagger = SequenceTagger(
        hidden_size=256,
        embeddings=embeddings,
        tag_dictionary=tag_dictionary,
        tag_type=tag_type,
        use_crf=False,
        use_rnn=False,
        reproject_embeddings=False,
        tag_format='BIO'
    )

2023-10-18 09:59:11,634 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason


In [77]:
# tagger : SequenceTagger = SequenceTagger(hidden_size=256,
#                                        embeddings=embeddings,
#                                        tag_dictionary=tag_dictionary,
#                                        tag_type=tag_type,
#                                        use_crf=True,
#                                          tag_format='BIO'
#                                          )
# print(tagger)

In [78]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.fine_tune(
        'resources/taggers/ner',
        learning_rate=5e-05,
        mini_batch_size=16,
        max_epochs=100,
        embeddings_storage_mode=None,
        weight_decay=0.0,
    )


2023-10-18 09:59:19,109 ----------------------------------------------------------------------------------------------------
2023-10-18 09:59:19,118 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28997, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_fea

100%|██████████| 2/2 [00:02<00:00,  1.46s/it]

2023-10-18 09:59:27,876 Evaluating as a multi-label problem: False
2023-10-18 09:59:27,998 DEV : loss 2.1662344932556152 - f1-score (micro avg)  0.0177
2023-10-18 09:59:28,027 ----------------------------------------------------------------------------------------------------


2023-10-18 09:59:28,790 epoch 2 - iter 1/6 - loss 2.21463094 - time (sec): 0.76 - samples/sec: 5717.61 - lr: 0.000005
2023-10-18 09:59:29,611 epoch 2 - iter 2/6 - loss 2.22208278 - time (sec): 1.58 - samples/sec: 6122.16 - lr: 0.000006
2023-10-18 09:59:30,321 epoch 2 - iter 3/6 - loss 2.17625425 - time (sec): 2.29 - samples/sec: 6019.00 - lr: 0.000007
2023-10-18 09:59:31,322 epoch 2 - iter 4/6 - loss 2.16112346 - time (sec): 3.29 - samples/sec: 5778.20 - lr: 0.000007
2023-10-18 09:59:32,100 epoch 2 - iter 5/6 - loss 2.14204746 - time (sec): 4.07 - samples/sec: 5735.46 - lr: 0.000008
2023-10-18 09:59:32,820 epoch 2 - iter 6/6 - loss 2.10249492 - time (sec): 4.79 - samples/sec: 5778.36 - lr: 0.000009
2023-10-18 09:59:32,827 ----------------------------------------------------------------------------------------------------
2023-10-18 09:59:32,829 EPOCH 2 done: loss 2.1025 - lr 0.000009


100%|██████████| 2/2 [00:05<00:00,  2.90s/it]

...

2023-10-18 10:07:54,865 ----------------------------------------------------------------------------------------------------
2023-10-18 10:07:54,870 Testing using last state of model ...


100%|██████████| 2/2 [00:00<00:00,  3.62it/s]

2023-10-18 10:07:55,437 Evaluating as a multi-label problem: False
2023-10-18 10:07:55,465 0.7469	0.8411	0.7912	0.6545


2023-10-18 10:07:55,468 
Results:
- F-score (micro) 0.7912
- F-score (macro) 0.7429
- Accuracy 0.6545

By class:
              precision    recall  f1-score   support

      Winner     0.5685    0.6871    0.6222       163
        Date     0.9847    1.0000    0.9923       129
 Nationality     0.8611    0.8857    0.8732       105
   Prizetype     0.8400    0.9545    0.8936        22
      Reason     0.2381    0.5556    0.3333         9

   micro avg     0.7469    0.8411    0.7912       428
   macro avg     0.6985    0.8166    0.7429       428
weighted avg     0.7728    0.8411    0.8032       428

2023-10-18 10:07:55,469 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7912087912087913,
 'dev_score_history': [0.017727128838239948,
  0.006535947712418301,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.39114391143911437,
  0.5900709219858156,
  0.6605263157894737,
  0.7310012062726176,
  0.7679425837320574,
  0.7864406779661016,
  0.825938566552901,
  0.8306636155606407,
  0.8209606986899564,
  0.8055853920515575,
  0.8288888888888888,
  0.8382022471910112,
  0.8306264501160093,
  0.8371559633027523,
  0.8316831683168318,
  0.8432919954904172,
  0.8372615039281706,
  0.8412698412698412,
  0.8353591160220994,
  0.8518518518518519,
  0.8460661345496009,
  0.8528389339513327,
  0.8364849833147943,
  0.8624708624708625,
  0.8587830080367395,
  0.8421052631578947,
  0.8571428571428572,
  0.8382022471910112,
  0.8652482269503546,
  0.8525714285714286,
  0.8624708624708625,
  0.8520179372197308,
  0.8564867967853043,
  0.8607888631090488,
  0.8555045871559633,
  0.8522336769759451,
  0.8509174311926606,
  0.8410372040586246,
  0.8454545454545455,
  0

In [79]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the trained model
model = SequenceTagger.load("resources/taggers/ner/final-model.pt")

# create example sentence
sentence = Sentence('Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. ')

# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

2023-10-18 10:08:04,163 SequenceTagger predicts: Dictionary with 11 tags: O, B-Winner, I-Winner, B-Date, I-Date, B-Nationality, I-Nationality, B-Prizetype, I-Prizetype, B-Reason, I-Reason
Sentence[79]: "Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer." → ["Barack Hussein Obama II"/Winner, "August 4, 1961"/Date, "American"/Nationality, "the United States"/Nationality, "he"/Winner, "Obama"/Winner, "U.S."/Nationality, "Illinois"/Nationality]


In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import RelationExtractor

relation_label_dict = corpus.make_label_dictionary(label_type="relation")
results_base_path = 'resources/relations/'
relation_model: RelationExtractor = RelationExtractor(
        embeddings=embeddings,
        label_dictionary=relation_label_dict,
        label_type="relation",
        entity_label_type="ner",
        # train_on_gold_pairs_only=True,
    )

trainer: ModelTrainer = ModelTrainer(relation_model, corpus)

trainer.train(
        results_base_path,
        learning_rate=5e-05,
        mini_batch_size=2,
        max_epochs=100,
        shuffle=True,
    )

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  )

In [3]:
from flair.data import Relation, Sentence
from flair.datasets import ColumnCorpus, DataLoader
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import RelationClassifier
from flair.models.relation_classifier_model import EncodedSentence
from flair.trainers import ModelTrainer

relation_label_dict = corpus.make_label_dictionary(label_type="relation")
relation_dictionary = corpus.make_label_dictionary("relation")
doc_embeddings = TransformerDocumentEmbeddings(model="distilbert-base-uncased", layers="-1", fine_tune=True)
relation_model: RelationClassifier = RelationClassifier(
    embeddings =doc_embeddings,
    label_dictionary = relation_dictionary,
    label_type="relation",
    entity_label_types="ner",
    entity_pair_labels={  # Define valid entity pair combinations, used as relation candidates
        ("Winner", "Nationality"),
        ("Winner", "Date"),
        ("Winner", "Prizetype"),
        ("Winner", "Reason"),

    },
    zero_tag_value='',
    allow_unk_tag=False,
)

2023-10-18 10:47:29,720 Computing label dictionary. Progress:


95it [00:00, 7786.20it/s]

2023-10-18 10:47:29,788 Dictionary created for label 'relation' with 7 values: has_won (seen 148 times), received_nobelprize_in (seen 144 times), received_nobelprize_for (seen 106 times), has_nationality (seen 94 times), born_on (seen 79 times), died_on (seen 41 times)
2023-10-18 10:47:29,790 Computing label dictionary. Progress:



95it [00:00, 7961.69it/s]

2023-10-18 10:47:29,813 Dictionary created for label 'relation' with 7 values: has_won (seen 148 times), received_nobelprize_in (seen 144 times), received_nobelprize_for (seen 106 times), has_nationality (seen 94 times), born_on (seen 79 times), died_on (seen 41 times)


In [5]:
trainer: ModelTrainer = ModelTrainer(model=relation_model, corpus=relation_model.transform_corpus(corpus))


In [6]:
trainer.fine_tune(
    'resources/relations/',
    max_epochs=10,
    learning_rate=4e-5,
    mini_batch_size=4,
    main_evaluation_metric=("macro avg", "f1-score"),
    shuffle=False,
)

2023-10-18 10:49:09,893 ----------------------------------------------------------------------------------------------------
2023-10-18 10:49:09,895 Model: "RelationClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768,

100%|██████████| 601/601 [00:19<00:00, 30.43it/s]


2023-10-18 10:52:51,072 Evaluating as a multi-label problem: False
2023-10-18 10:52:51,103 DEV : loss 0.2600390315055847 - f1-score (macro avg)  0.4547
2023-10-18 10:52:51,303 ----------------------------------------------------------------------------------------------------
2023-10-18 10:53:14,031 epoch 2 - iter 282/2828 - loss 0.08470342 - time (sec): 22.73 - samples/sec: 49.63 - lr: 0.000018
2023-10-18 10:53:35,938 epoch 2 - iter 564/2828 - loss 0.11936852 - time (sec): 44.63 - samples/sec: 50.55 - lr: 0.000019
2023-10-18 10:53:58,245 epoch 2 - iter 846/2828 - loss 0.14879789 - time (sec): 66.94 - samples/sec: 50.55 - lr: 0.000021
2023-10-18 10:54:17,938 epoch 2 - iter 1128/2828 - loss 0.17760354 - time (sec): 86.63 - samples/sec: 52.08 - lr: 0.000022
2023-10-18 10:54:34,291 epoch 2 - iter 1410/2828 - loss 0.19915226 - time (sec): 102.99 - samples/sec: 54.76 - lr: 0.000024
2023-10-18 10:54:56,772 epoch 2 - iter 1692/2828 - loss 0.19511761 - time (sec): 125.47 - samples/sec: 53.94 -

100%|██████████| 601/601 [00:20<00:00, 28.79it/s]


2023-10-18 10:56:29,093 Evaluating as a multi-label problem: False
2023-10-18 10:56:29,139 DEV : loss 0.2930014729499817 - f1-score (macro avg)  0.5596
2023-10-18 10:56:29,533 ----------------------------------------------------------------------------------------------------
2023-10-18 10:56:53,683 epoch 3 - iter 282/2828 - loss 0.06538009 - time (sec): 24.15 - samples/sec: 46.71 - lr: 0.000034
2023-10-18 10:57:15,718 epoch 3 - iter 564/2828 - loss 0.10247270 - time (sec): 46.18 - samples/sec: 48.85 - lr: 0.000035
2023-10-18 10:57:38,705 epoch 3 - iter 846/2828 - loss 0.13728137 - time (sec): 69.17 - samples/sec: 48.92 - lr: 0.000037
2023-10-18 10:57:58,235 epoch 3 - iter 1128/2828 - loss 0.16489833 - time (sec): 88.70 - samples/sec: 50.87 - lr: 0.000038
2023-10-18 10:58:14,338 epoch 3 - iter 1410/2828 - loss 0.17662764 - time (sec): 104.80 - samples/sec: 53.82 - lr: 0.000040
2023-10-18 10:58:36,217 epoch 3 - iter 1692/2828 - loss 0.17749560 - time (sec): 126.68 - samples/sec: 53.43 -

...

2023-10-18 12:19:49,444 Evaluating as a multi-label problem: False
2023-10-18 12:19:49,506 DEV : loss 0.24972154200077057 - f1-score (macro avg)  0.8415


2023-10-18 12:19:55,172 ----------------------------------------------------------------------------------------------------
2023-10-18 12:19:55,174 Testing using last state of model ...


100%|██████████| 640/640 [00:28<00:00, 22.51it/s]

2023-10-18 12:20:23,675 Evaluating as a multi-label problem: False
2023-10-18 12:20:23,718 0.3068	0.7778	0.44	0.9246
2023-10-18 12:20:23,720 
Results:
- F-score (micro) 0.44
- F-score (macro) 0.7026
- Accuracy 0.9246

By class:
                         precision    recall  f1-score   support

                has_won     0.1118    0.7600    0.1949        25
 received_nobelprize_in     0.4375    0.6364    0.5185        22
        has_nationality     0.9444    0.8500    0.8947        20
received_nobelprize_for     0.8462    0.7857    0.8148        14
                born_on     0.9091    0.8333    0.8696        12
                died_on     0.8571    1.0000    0.9231         6

              micro avg     0.3068    0.7778    0.4400        99
              macro avg     0.6843    0.8109    0.7026        99
           weighted avg     0.5980    0.7778    0.6218        99

2023-10-18 12:20:23,722 -----------------------------------------------------------------------------------------------

{'test_score': 0.7025973517964365,
 'dev_score_history': [0.4546792513197888,
  0.5595776122091912,
  0.7239617994110841,
  0.6931253643215994,
  0.7536805153799054,
  0.7797558591421692,
  0.6568728124778177,
  0.6342767531364023,
  0.7375155438109434,
  0.7977278898331531,
  0.7687416749051122,
  0.7738281501340151,
  0.7824181252703825,
  0.7854776009687052,
  0.8015384314599725,
  0.8176726434524096,
  0.8175346392737698,
  0.7852272881684647,
  0.81995179494146,
  0.8176457654478074,
  0.8287365298234864,
  0.8452458652458654,
  0.8454549898735945,
  0.8415477656072378,
  0.8415477656072378],
 'train_loss_history': [0.39616341553548157,
  0.19209932283059444,
  0.17207473808714666,
  0.1447471973461033,
  0.1152143246691582,
  0.09772202758199716,
  0.07885806364838581,
  0.06855294551654241,
  0.05728943422738625,
  0.04305166787002876,
  0.03604615783428277,
  0.03029910868300059,
  0.0347964082951069,
  0.02312743476363078,
  0.020256069530330035,
  0.01648998673432721,
  0.013

In [8]:
loaded_model: RelationClassifier = RelationClassifier.load('resources/relations/final-model.pt')